In [21]:
from numpy import linalg as LA
import matplotlib.pyplot as plt
import circle_fit as cf
import numpy as np
import cv2
import os

## Get subpixel edge locations

In [25]:
def get_sample_value(coord, img):
    center = np.around(coord)
    
    x, y = center[0], center[1]
    neighbor_val = []
    neighbor_dist = []
    
    ## 取的9格的距離跟值
    for i in range(-1,2):
        for j in range(-1,2):
            dist = LA.norm((center - np.asrray([j,i])) - coord)
            neighbor_val.append(img[y+i, x+j])
            neighbor_dist.apend(dist)
    
    ## 依照距離反比給權重
    neighbor_val = (1 - neighbor_val)/8
    
    value = np.dot(neighbor_val, neighbor_dist)
    return value


def get_subpixel_edge(center, edge_points, img):
    sp_edge = []
    
    for point in edge_points:
        ## 取得中心到邊緣的單位向量
        c2e_Vector = point - center
        c2e_Vector = vector/LA.norm(vector)
        
        
        sample_points = []
        for i in range(-3,3.05,0.2):
            ## 生出要擬合 S 曲線的點
            sample_location = point + i*c2e_Vector
            sample_value = get_sample_value(sample_location, img)
            
            
        ## 擬合 S 曲線
            
            
        break
    
    return sp_edge

In [31]:
a = np.asarray([0.1, 0.5])
b = np.asarray([5, 10])

np.dot(a,b)

5.5

In [26]:
def image_processing(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5,5), 0)
    nlm  = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
    _, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    edge = cv2.Canny(otsu, 0, 20)
    
    return blur, nlm, otsu, edge

def get_edge_points(edge_img):
    edge_points = [[j,i] for i, r in enumerate(edge_img) for j, c in enumerate(r) if c == 255]

    return edge_points

In [27]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
file = "10_mean_80.bmp"
img = cv2.imread(file, 0)
blur, nlm, otsu, edge = image_processing(img)

edge_points = get_edge_points(edge)
cx,cy,r,_ = cf.least_squares_circle((edge_points))

sp_edge = get_subpixel_edge(np.asarray([cx,cy]), edge_points, img)


print("{}, {}".format(cx, cy))

[-0.06555615 -0.99784888]
203.43340624689515, 99.48214409869645
